In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# display all columns
pd.set_option('display.max_columns', None)

In [ ]:
data = pd.read_csv("./data/2018-2020_add_sales_info.csv", encoding='utf-8')

In [ ]:
data

In [ ]:
"""
    加入保額、保費的分群資訊 (配合客戶資料, 越大金額越小)
    5 : 1w 以下
    4 : 1w ~ 10w
    3 : 10w ~ 100w
    2 : 100w ~ 500w
    1 : 500w 以上
"""

def fee_range_info(data):
    fee = data['保費']
    if fee<10001: return 5
    elif fee>10000 and fee<100001: return 4
    elif fee>100000 and fee<1000001: return 3
    elif fee>1000000 and fee<5000001: return 2
    else: return 1
    
def count_range_info(data):
    fee = data['保額']
    if fee<10001: return 5
    elif fee>10000 and fee<100001: return 4
    elif fee>100000 and fee<1000001: return 3
    elif fee>1000000 and fee<5000001: return 2
    else: return 1
    
data.loc[:, '保費區間'] = data.apply(fee_range_info, axis=1)
data.loc[:, '保額區間'] = data.apply(count_range_info, axis=1)

In [ ]:
"""
    幣別 ('TWD': 0, 'USD': 1)
"""
df_dollartype = pd.get_dummies(data['幣別'])
df_dollartype = df_dollartype.drop('USD', 1)

data = data.drop('幣別', 1)
data = pd.concat([data, df_dollartype], axis=1)

In [ ]:
"""
    Data Label ('Y':1, 'N':0)
"""
data['是否已受理'] = data['是否已受理'].map({'Y':1, 'N':0})

In [ ]:
# 看 timespan 的數值分布

timespan = data['Timespan']
plt.hist(timespan, bins=[0, 1, 2, 3, 4, 5, 6, 7, 30], color='lightblue')

In [ ]:
"""
Ver. 1:
    保費, 保額直接做 normalize
"""
# data_fee = data['保費']
# fee_min = data['保費'].min()
# fee_max = data['保費'].max()
# data['保費'] = (data['保費']-fee_min)/(fee_max-fee_min)

# data_fee = data['保額']
# fee_min = data['保額'].min()
# fee_max = data['保額'].max()
# data['保額'] = (data['保額']-fee_min)/(fee_max-fee_min)

In [ ]:
"""
Ver. 3:
    保費、保額先做分群再取 z-score

各個區間
a : 1w 以下
b : 1w ~ 10w
c : 10w ~ 100w
d : 100w ~ 500w
e : 500w 以上
"""

# 保費
a_fee = data[data['保費']<=10000]
b_fee = data[(data['保費']>10000) & (data['保費']<=100000)]
c_fee = data[(data['保費']>100000) & (data['保費']<=1000000)]
d_fee = data[(data['保費']>1000000) & (data['保費']<=5000000)]
e_fee = data[data['保費']>5000000]

a_fee['保費'] = (a_fee['保費'] - 2664.5) / 2217.5
b_fee['保費'] = (b_fee['保費'] - 32781.7) / 22002.6
c_fee['保費'] = (c_fee['保費'] - 315331.1) / 228632.5
d_fee['保費'] = (d_fee['保費'] - 1712876.7) / 881441.3
e_fee['保費'] = (e_fee['保費'] - 14955516.5) / 20320226.8

data = pd.concat([a_fee, b_fee, c_fee, d_fee, e_fee])


# 保額
a_count = data[data['保額']<=10000]
b_count = data[(data['保額']>10000) & (data['保額']<=100000)]
c_count = data[(data['保額']>100000) & (data['保額']<=1000000)]
d_count = data[(data['保額']>1000000) & (data['保額']<=5000000)]
e_count = data[data['保額']>5000000]

a_count['保額'] = (a_count['保額'] - 2673.8) / 2554.8
b_count['保額'] = (b_count['保額'] - 43313.8) / 22360.3
c_count['保額'] = (c_count['保額'] - 598427.3) / 315196.4
d_count['保額'] = (d_count['保額'] - 1857489.3) / 718524.4
e_count['保額'] = (e_count['保額'] - 11439169.0) / 9792245.2

data = pd.concat([a_count, b_count, c_count, d_count, e_count])


# Reset index
data = data.reset_index(inplace=False, drop=False)

# Remove unnamed columns
data = data.drop(['Unnamed: 0', 'index'], axis=1)

In [ ]:
data

In [ ]:
"""
    整理 "商品分類_三標":

"""
type_dict = {'A&H(健康意外險)':'A&H', 'SP(躉繳)':'SP', '躉繳':'SP', 'RP金流(壽險期繳金流型)':'RP1', '終身壽險':'RP1', 'RP保障(壽險期繳保障型)':'RP2'}

data['商品分類_三標'] = data['商品分類_三標'].replace(type_dict)
# df_category = pd.get_dummies(data['商品分類_三標'])
# data = data.drop('商品分類_三標', 1)
# data = pd.concat([data,df_category], axis=1)

In [ ]:
data

In [ ]:
# # 商品分類_保障項目
# df_type = pd.get_dummies(data['商品分類_保障項目'])
# data = data.drop('商品分類_保障項目', 1)
# data = pd.concat([data,df_type], axis=1)

In [ ]:
"""
    Timespan (去除 NaN 和 0,1 )
    
    ps. 這格要在Z-score後面!!
"""

zero_or_one_rows = [i for i,x in enumerate(data['Timespan']) if data['Timespan'][i]==0 or data['Timespan'][i]==1]

data = data.fillna('null')
nan_rows = [i for i,x in enumerate(data['Timespan']) if data['Timespan'][i]=='null']

data = data.drop(zero_or_one_rows)
data = data.drop(nan_rows)

In [ ]:
"""
    去除多餘欄位
"""

data = data.drop(['資料月份', '建議書_最後異動日', '商品名稱', '是否完成E受理', 'e受理序號dummy'], axis=1)

In [ ]:
data

In [ ]:
"""
    將資料分成 新客戶、舊客戶
"""

data_old = data[(data['財富指標']!='null') & (data['客戶忠誠度']!='null') & (data['客戶分群(NEW)']!='null')]
data_new = data[(data['財富指標']=='null') & (data['客戶忠誠度']=='null') & (data['客戶分群(NEW)']=='null')]

In [ ]:
print('all: ',len(data))
print('old: ',len(data_old))
print('new: ',len(data_new))

In [ ]:
"""
    新舊客戶一起做訓練
"""

# data.to_csv('./data/preprocessing_v6_all.csv')

In [ ]:
"""
Ver.4
    新舊客戶分開訓練
"""

# data_new.to_csv('./data/preprocessing_v6_new.csv')
# data_old.to_csv('./data/preprocessing_v6_old.csv')

In [ ]:
"""
Ver.4、Ver.5 "加入業務員成交率" 所使用的資料
    
"""
# data_old = data_old.drop(['index', 'Column1'], axis=1)
# data_old.to_csv('./data/2018-2020_old_add_sales_info.csv')

In [ ]:
data_old

In [ ]:
data_new